# Backups

***You will need to have followed the steps in the 00-CreateContainers notebook to use this notebook***

We have written a book which will give you a brilliant introduction to dbatools. It's called dbatools in a Month of Lunches and you can find it at https://beard.media/book

dbatools is **awesome** at performing backups :-)

The next block sets the variables for tje instances and folder paths for this Notebook and checks the connection - Refer to the first notebook for any issues

In [1]:
$FolderPath = $Env:USERPROFILE + '\Documents\dbatoolsdemo'
$SqlInstances = 'localhost,15592', 'localhost,15593'
$SqlCredential = Import-Clixml -Path $FolderPath\sqladmin.cred
Write-Output " Creating connection to the containers"
try {
    $SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
    $SQL2 = Connect-DbaInstance -SqlInstance $SqlInstances[1] -SqlCredential $SqlCredential
    Write-Output "We have a connection to the containers"

}
catch {
    Write-Output "You haven't got a connection to the containers - Either they are still upgrading in which case try again in 30 seconds or the containers have not come up correctly"
    Write-Output "Make sure the containers are running - the code is below in a block for you"
    Write-Output "docker ps -a"
    Write-Output "If they are read the logs - the code is below in a block for you"
    Write-Output "docker logs dbatools_SQL2019_1"
    Write-Output "docker logs dbatools_SQL2019-1_1"
}

Creating connection to the containers

We have a connection to the containers

## Investigate a directory using the SQL Service Account

We can investigate the file system from the viewpoint of the SQL Service Account using `Get-DbaFile` We are going to use that to show the files in the directory but this can be really useful when investigating errors with file system access, for example when backups fail or when reading from shares. This command will try ot access the path from teh SQL Instance using the SQL Account which means that you can troubleshoot the error and also that users do not need access to directories holding backups or client files for example  
If we look in the Directory that we created in the create container notebook using the command below

In [2]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\dummyfile.txt 01a539677f71

You can see that all there is in the directory is a dummy file that we created earlier when we set up the directories  

## Check the last time that the databases were backed up

Knowing the last time a database was backed up is often useful information and with dbatools it can be retrieved easily. (NB - I use Warning action silently continue to hide the backups from multiple forks warnings that I have created when I created the container and havent fixed yet!)

In [3]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBacku

p

------------ ------------ ----------- -------- -------------- -------------- ------------

localhost MSSQLSERVER 01a539677f71 AdventureWorks2017 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 01a539677f71 master 2019-12-21 14:41:56.000

...

localhost MSSQLSERVER e5e6a324ee39 model 2019-12-21 14:41:56.000

localhost MSSQLSERVER e5e6a324ee39 msdb 2019-12-21 14:41:57.000

localhost MSSQLSERVER e5e6a324ee39 Northwind 2019-12-21 14:42:01.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER e5e6a324ee39 pubs 2019-12-21 14:42:02.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER e5e6a324ee39 WideWorldImporters 2019-12-21 14:42:03.000 2019-12-21 14:42:02.000

The databases were last backed up on the 21st December 2019. Thats not so clever!

## Docker requires permissions

You will get errors for the command below if the account that you have used to allow Docker to access your drives does not have permissions to the folder that we have created. [You can see the instructions here](https://docs.docker.com/docker-for-windows/)

You should add the docker account with full control over your dbatoolsdemo directory in your User Profile directory if you get errors below.

![DockerCompose](.\images\dbatoolsdemopermissions.png )

Its annoying but you can give the docker account modify permissions to your documents folder and it will inherited

![DockerCompose](.\images\documentsdirectorypermissions.png )



## Perform a backup of the entire instance

With one line of code we can quickly backup an entire instance.

In [4]:
Backup-DbaDatabase  -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

01a539677f71 model Full 2.59 MB Disk 2020-03-05 09:56:42.000 00:00:01 2020-03-05 09:56:43.000

01a539677f71 msdb Full 16.09 MB Disk 2020-03-05 09:56:43.000 00:00:01 2020-03-05 09:56:44.000

01a539677f71 Northwind Full 6.71 MB Disk 2020-03-05 09:56:44.000 00:00:00 2020-03-05 09:56:44.000

01a539677f71 pubs Full 4.46 MB Disk 2020-03-05 09:56:45.000 00:00:00 2020-03-05 09:56:45.000

01a539677f71 WideWorldImporters Full 448.10 MB Disk 2020-03-05 09:56:46.000 00:00:08 2020-03-05 09:56:54.000

All of the databases are backed up - Lets have a look in the directory

In [5]:
Get-DbaFile -SqlInstance $SQL1 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL1

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL1\AdventureWorks2017_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\dummyfile.txt 01a539677f71

/var/opt/mssql/backups/SQL1\master_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\model_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\msdb_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\Northwind_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\pubs_202003050956.bak 01a539677f71

/var/opt/mssql/backups/SQL1\WideWorldImporters_202003050956.bak 01a539677f71

If you want to have a look in the mounted volume on your laptop 

In [6]:
Get-ChildItem "$FolderPath\SQL1"

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL1

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 05/03/2020 09:56 217178112 AdventureWorks2017_202003050956.bak

-a--- 05/03/2020 09:02 0 dummyfile.txt

...

-a--- 05/03/2020 09:56 2744320 model_202003050956.bak

-a--- 05/03/2020 09:56 16896000 msdb_202003050956.bak

-a--- 05/03/2020 09:56 7065600 Northwind_202003050956.bak

-a--- 05/03/2020 09:56 4706304 pubs_202003050956.bak

-a--- 05/03/2020 09:56 469901312 WideWorldImporters_202003050956.bak

## Backup to a directory for each database

Rob - I hear you cry, I have 2000 databases, I dont want them all backing up to the same directory - Can dbatools help me there ?

Sure, just add the `-CreateFolder` switch.

Lets demonstrate with SQL2

Theres nothing up my sleeves, lets look at the SQL2 folder


In [7]:
Get-DbaFile -SqlInstance $SQL2 -SqlCredential $SqlCredential -Path /var/opt/mssql/backups/SQL2

Filename SqlInstance

-------- -----------

/var/opt/mssql/backups/SQL2\dummyfile.txt e5e6a324ee39

Only the dummy file! 

### Backup the databases

In [8]:
Backup-DbaDatabase -SqlInstance $SQL2 -SqlCredential $SqlCredential -CreateFolder -Path /var/opt/mssql/backups/SQL2 -CompressBackup -WarningAction SilentlyContinue

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

PROGRESS: Complete

...

e5e6a324ee39 model Full 2.59 MB Disk 2020-03-05 09:59:04.000 00:00:00 2020-03-05 09:59:04.000

e5e6a324ee39 msdb Full 16.09 MB Disk 2020-03-05 09:59:05.000 00:00:00 2020-03-05 09:59:05.000

e5e6a324ee39 Northwind Full 6.71 MB Disk 2020-03-05 09:59:05.000 00:00:01 2020-03-05 09:59:06.000

e5e6a324ee39 pubs Full 4.46 MB Disk 2020-03-05 09:59:06.000 00:00:00 2020-03-05 09:59:06.000

e5e6a324ee39 WideWorldImporters Full 448.10 MB Disk 2020-03-05 09:59:07.000 00:00:04 2020-03-05 09:59:11.000

## have a look in the directory

In [10]:
Get-ChildItem "$FolderPath\SQL2" -Recurse

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2

Mode LastWriteTime Length Name

---- ------------- ------ ----

d---- 05/03/2020 09:59 AdventureWorks2017

d---- 05/03/2020 09:59 master

...

-a--- 05/03/2020 09:59 520192 pubs_202003050959.bak

Directory: C:\Users\mrrob\Documents\dbatoolsdemo\SQL2\WideWorldImporters

Mode LastWriteTime Length Name

---- ------------- ------ ----

-a--- 05/03/2020 09:59 128180224 WideWorldImporters_202003050959.bak

As you can see we have the databases backed up, each in their own directory :-)

## Double check all the instances

dbatoos works with multiple instances whereever there is a `-SqlInstance` parameter

Lets check the last time that these databases were backed up on both of the containers

In [11]:
Get-DbaLastBackup -SqlInstance $SQL1,$SQL2 -SqlCredential $SqlCredential -WarningAction SilentlyContinue | Format-Table 

ComputerName InstanceName SqlInstance Database LastFullBackup LastDiffBackup LastLogBacku

p

------------ ------------ ----------- -------- -------------- -------------- ------------

localhost MSSQLSERVER 01a539677f71 AdventureWorks2017 2020-03-05 09:56:41.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER 01a539677f71 master 2020-03-05 09:56:42.000

...

localhost MSSQLSERVER e5e6a324ee39 model 2020-03-05 09:59:04.000

localhost MSSQLSERVER e5e6a324ee39 msdb 2020-03-05 09:59:05.000

localhost MSSQLSERVER e5e6a324ee39 Northwind 2020-03-05 09:59:06.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER e5e6a324ee39 pubs 2020-03-05 09:59:06.000 2019-12-21 14:42:00.000

localhost MSSQLSERVER e5e6a324ee39 WideWorldImporters 2020-03-05 09:59:11.000 2019-12-21 14:42:02.000

# We want to use Ola Hallengren for our backups

Thats ok, dbatools supports many of the excellent community tooling including Ola Hallengrens solution.

You can install Ola Hallengren like this (except in containers)

````
$installDbaMaintenanceSolutionSplat = @{
    SqlInstance = $sql1
    InstallJobs = $true
    ReplaceExisting = $true
    LogToTable = $true
    OutputFileDirectory = '/var/opt/mssql/backups/SQL1'
    BackupLocation = '/var/opt/mssql/backups/SQL1'
    CleanupTime = 72
    Solution = 'All'
    Database = 'master'
}
Install-DbaMaintenanceSolution @installDbaMaintenanceSolutionSplat -Verbose
````

Lets have a look at the Agent Jobs on the Instance

In [12]:
Get-DbaAgentJob -SqlInstance $sql1 |Select Name

Name

----

CommandLog Cleanup

DatabaseBackup - SYSTEM_DATABASES - FULL

DatabaseBackup - USER_DATABASES - DIFF

...

DatabaseIntegrityCheck - USER_DATABASES

IndexOptimize - USER_DATABASES

sp_delete_backuphistory

sp_purge_jobhistory

The Beard is Important

Those look like Ola Jobs to me :-)

Lets run the jobs to back up the system and user databases

In [13]:
$Jobs = 'DatabaseBackup - SYSTEM_DATABASES - FULL' ,'DatabaseBackup - USER_DATABASES - FULL'
Start-DbaAgentJob -SqlInstance $sql1 -Job $Jobs

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : 01a539677f71

Name : DatabaseBackup - SYSTEM_DATABASES - FULL

Category : Database Maintenance

...

LastRunDate : 21/12/2019 14:42:00

LastRunOutcome : Succeeded

HasSchedule : False

OperatorToEmail :

CreateDate : 21/12/2019 14:35:24

Check the status of the jobs

In [14]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaAgentJob -SqlInstance $sql1 -Job $Jobs | Select SqlInstance, Name, CurrentRunStatus, LastRunOutCome

SqlInstance Name CurrentRunStatus LastRunOutcome

----------- ---- ---------------- --------------

01a539677f71 DatabaseBackup - SYSTEM_DATABASES - FULL Idle Succeeded

01a539677f71 DatabaseBackup - USER_DATABASES - FULL Idle Succeeded

When the jobs have finished, lets have a look at the files.
We need to alter the file path to include the current container name from the results above

In [16]:
Get-DbaFile -SqlInstance $sql1 -Path '/var/opt/mssql/data/01a539677f71/AdventureWorks2017/FULL'

Filename SqlI

nsta

nce

-------- ----

/var/opt/mssql/data/01a539677f71/AdventureWorks2017/FULL\01a539677f71_AdventureWorks2017_FULL_20200305_100242.bak 01a…

# Restores

Backups are all very well and good but what about restores?

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason

In [17]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : 01a539677f71

Database : AdventureWorks2017

Status : Dropped

...

ComputerName : localhost

InstanceName : MSSQLSERVER

SqlInstance : 01a539677f71

Database : pubs

Status : Dropped

In [18]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

All the databases have gone - How easy it is to restore them?

This easy :-)

One line of code

## First lets restore from the backups we took with dbatools in, with all files in the one directory

In [19]:
Write-Output "Starting Restoring"
Restore-DbaDatabase  -SqlInstance $SQL1 -Path /var/opt/mssql/backups/SQL1 
Write-Output "Finished"

Starting Restoring

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

That was easy - WHat about if I have used Ola Hallengren?

AGAIN

**NEVER EVER DO THIS IN PRODUCTION**
unless you need to delete all of your user databases for some reason


In [ ]:
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Remove-DbaDatabase -Confirm:$false

## Restore from Ola Halengren Backups 

- Use the path from the above files check with the new container name 

In [ ]:
Write-Output "Starting Restoring"
Restore-DbaDatabase -SqlInstance $SQL1 -Path /var/opt/mssql/data/ca2f813564a4 -MaintenanceSolutionBackup
Write-Output "Finished"

In [ ]:
$SQL1 = Connect-DbaInstance -SqlInstance $SqlInstances[0] -SqlCredential $SqlCredential 
Get-DbaDatabase -SqlInstance $SQL1 -ExcludeAllSystemDb | Select Name

Thats all very well and good

## When were the databases restored?

and which backup was used?

In [ ]:
Get-DbaDbRestoreHistory -SqlInstance $SQL1 -Last

# Don't forget to clean up

Now you can use these containers to run the rest of the notebooks - Don't forget to run the Clean Up Containers Notebook at the end to clean up